In [1]:
import os 
import sys
import glob
import time
from tqdm import tqdm

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 

import MDAnalysis as mda
from MDAnalysis.analysis import distances
from MDAnalysis.analysis.rms import RMSD

In [2]:
runs = sorted(glob.glob("../../traj_save/*/*pdb"))

In [3]:
df = []

for run in tqdm(runs):
    pdb_file = run
    dcd_file = run.replace("pdb", "dcd")
    ref_pdb = pdb_file # [:-7] + '000.pdb'
#     print(ref_pdb)
#     continue

    run_base = os.path.basename(run)[:-4]
#     run_label = run_base.split("_")[2]
    
    mda_traj = mda.Universe(pdb_file, dcd_file)
    ref_traj = mda.Universe(ref_pdb)
    
    rmsd_inst = RMSD(
            mda_traj, ref_traj,
            select='protein and name CA',
            groupselections=[
                    "protein and name CA and segid B",   # CORE
                    "protein and name CA and segid A",                                   # LID
                    ],
            # verbose=1
                    )
    rmsd_inst.run()
        
    df.append({'sys_name': run_base,
               'time_frame': rmsd_inst.rmsd[:,1],
               'rmsd_all': rmsd_inst.rmsd[:,2], 
               'rmsd_nsp10': rmsd_inst.rmsd[:,3], 
               'rmsd_nsp16': rmsd_inst.rmsd[:,4]})

 11%|█▏        | 12/106 [00:51<06:48,  4.34s/it]/homes/heng.ma/miniconda3/envs/MD_ff/lib/python3.6/site-packages/MDAnalysis/analysis/rms.py:255: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(np.sum((a - b) ** 2) / N)
100%|██████████| 106/106 [04:38<00:00,  2.63s/it]


In [4]:
df = pd.DataFrame(df)
df.head()

,sys_name,time_frame,rmsd_all,rmsd_nsp10,rmsd_nsp16
0,comp_0,"[0.0, 1.0000000328495406, 2.0000000656990813, ...","[0.0005033500343547033, 1.128382942405615, 1.1...","[0.000508360874832082, 1.4329209931079394, 1.6...","[0.0005014563430159343, 0.9852074835504431, 0...."
1,comp_1,"[0.0, 1.0000000328495406, 2.0000000656990813, ...","[0.0005018848402518594, 0.9914718976681515, 1....","[0.0005055609301283358, 1.2652129825936829, 1....","[0.0005005624808340656, 0.8621805325663472, 0...."
2,comp_2,"[0.0, 1.0000000328495406, 2.0000000656990813, ...","[0.0004995165125159671, 1.0053374442466876, 1....","[0.0005036704968548937, 1.188780600891894, 1.3...","[0.000497981015218141, 0.9244193827179352, 1.1..."
3,comp_sah_0,"[0.0, 1.0000000328495406, 2.0000000656990813, ...","[0.0004911495038199802, 1.0124807557771016, 1....","[0.00048357470470672826, 1.1705811276657037, 1...","[0.0004939909070037616, 0.9440421327698263, 0...."
4,comp_sah_1,"[0.0, 1.0000000328495406, 2.0000000656990813, ...","[0.000494302682957947, 1.0588426217448117, 1.1...","[0.0005172342423416062, 1.4291491339915676, 1....","[0.0004851089083457245, 0.8739067258251104, 1...."


In [5]:
df.to_pickle('rmsd_nsp10-16_all.pkl')